## Imports
---

In [1]:
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews as mr
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\albert\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already u

In [2]:
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd

import matplotlib.pyplot as plt



import re #Regexs
import os #Acces a fitxers

c:\Users\Albert\Anaconda3\envs\myenv\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Exploració de dades
---

In [3]:
# Obtener las categorías disponibles
categories = mr.categories()
print(categories)

# Obtener las IDs de las revisiones de películas
file_ids = mr.fileids()
print(file_ids[8])



['neg', 'pos']
neg/cv008_29326.txt


In [4]:
mr.raw(file_ids[0])

'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it\'s simply too jumbled . \nit starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience membe

## Preprocessing
---

In [5]:
def preprocessing(text):
    
    stop_words = set(stopwords.words('english'))
    
    # Tokenización
    text = re.sub(r'[^\w\s]', '', text) # Elimina signos de puntuación
    text = re.sub(r'\d+', '', text) # Elimina números
    text = text.lower() # Convierte a minúsculas
    text = re.sub(r'[\s\n]+', ' ', text) # Elimina espacios en blanco sobrantes excepto \n
    text = re.sub(r' +', ' ', text) # Elimina espacios en blanco largos
    text = re.sub(r'_', '', text) # Elimina barras bajas

    tokens = nltk.word_tokenize(text)
    
    # Filtrar stop words
    tokens = [token for token in tokens if token not in stop_words]
    
    return ' '.join(tokens)

In [6]:
X, y = [], []

# Preprocesamiento de todas las reviews
for elem in file_ids:
    X.append(preprocessing(mr.raw(elem)))
    y.append(mr.categories(elem)[0])


## Train-Test division
---

In [7]:
y_num = []

for elem in y:
    if elem == 'neg':
        elem = 0
    else:
        elem = 1
    y_num.append(elem)

In [8]:
#dividimos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y_num, test_size=0.2, random_state=69)

## Representació
---

- Lematitzar 
- Vectoritzar (fit en el train, transform en el train i en el test) (counts >= 5)
- Modelos


In [9]:
wnl = nltk.stem.WordNetLemmatizer()

def lemmatize(p):
  d = {'NN': 'n', 'NNS': 'n', 
       'JJ': 'a', 'JJR': 'a', 'JJS': 'a', 
       'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v', 
       'RB': 'r', 'RBR': 'r', 'RBS': 'r'}
  if p[1] in d:
    return wnl.lemmatize(p[0], pos=d[p[1]])
  return p[0]

def lem_func(text):
    tokens = nltk.word_tokenize(text)
    tokens = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

In [10]:
X_train_lemmatized = []

for text in X_train:
    X_train_lemmatized.append(lem_func(text))

In [11]:
cv = CountVectorizer(min_df=5)
cv.fit(X_train_lemmatized)

def vect(text):
    a = cv.transform(text)
    return pd.DataFrame(a.toarray(), columns=cv.get_feature_names_out())


In [12]:
X_train_cv = vect(X_train_lemmatized)

In [13]:
X_train_cv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Columns: 9627 entries, aaron to zwicks
dtypes: int64(9627)
memory usage: 117.5 MB


In [14]:
#Eliminar stop words del dataframe